In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
orders = pd.read_csv("../train/orders.csv")

C:\Users\dybsb\AppData\Local\Temp\ipykernel_2732\33406138.py:1: DtypeWarning: Columns (15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  orders = pd.read_csv("../train/orders.csv")


In [6]:
orders.head()

,order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,...,driver_accepted_time,ready_for_pickup_time,picked_up_time,delivered_time,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR
0,163923.0,KL09J9N,6.0,10.1,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,8/1/2024 5:30,84,8/2/2024 5:33,0,Work,KL09J9N X 0 X 84
1,163924.0,H5LGGFX,3.0,8.4,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,8/1/2024 5:30,78,8/2/2024 5:34,0,Home,H5LGGFX X 0 X 78
2,163925.0,CYLZB6T,4.0,15.0,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,8/1/2024 5:30,4,8/2/2024 5:35,0,Work,CYLZB6T X 0 X 4
3,163929.0,4YKUKYN,7.0,27.2,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,8/1/2024 5:30,157,8/2/2024 5:39,0,Home,4YKUKYN X 0 X 157
4,163930.0,WDNU30K,1.0,6.5,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,8/1/2024 5:30,160,8/2/2024 5:39,0,Home,WDNU30K X 0 X 160


In [5]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135303 entries, 0 to 135302
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   order_id                        135233 non-null  float64
 1   customer_id                     135303 non-null  object 
 2   item_count                      128378 non-null  float64
 3   grand_total                     135303 non-null  float64
 4   payment_mode                    135303 non-null  int64  
 5   promo_code                      4305 non-null    object 
 6   vendor_discount_amount          135303 non-null  float64
 7   promo_code_discount_percentage  65880 non-null   float64
 8   is_favorite                     100108 non-null  object 
 9   is_rated                        135303 non-null  object 
 10  vendor_rating                   45220 non-null   float64
 11  driver_rating                   135303 non-null  int64  
 12  deliverydistance

In [7]:
orders.shape

(135303, 26)

In [8]:
# Missing value check
print("\nMissing values:")
orders.isnull().sum()


Missing values:


order_id                              70
customer_id                            0
item_count                          6925
grand_total                            0
payment_mode                           0
promo_code                        130998
vendor_discount_amount                 0
promo_code_discount_percentage     69423
is_favorite                        35195
is_rated                               0
vendor_rating                      90083
driver_rating                          0
deliverydistance                       0
preparationtime                    55560
delivery_time                     130180
order_accepted_time                48348
driver_accepted_time               88845
ready_for_pickup_time              51054
picked_up_time                     51438
delivered_time                     49562
delivery_date                      99759
vendor_id                              0
created_at                             0
LOCATION_NUMBER                        0
LOCATION_TYPE   

In [9]:
# Convert time columns to datetime
time_cols = [
    'order_accepted_time', 'driver_accepted_time', 'ready_for_pickup_time',
    'picked_up_time', 'delivered_time', 'created_at', 'delivery_date'
]
for col in time_cols:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')

In [10]:
#  Calculate time durations
orders['preparation_time'] = (orders['ready_for_pickup_time'] - orders['order_accepted_time']).dt.total_seconds() / 60
orders['driver_wait_time'] = (orders['picked_up_time'] - orders['ready_for_pickup_time']).dt.total_seconds() / 60
orders['delivery_duration'] = (orders['delivered_time'] - orders['picked_up_time']).dt.total_seconds() / 60
orders['total_delivery_time'] = (orders['delivered_time'] - orders['order_accepted_time']).dt.total_seconds() / 60

In [14]:
orders.head()

,order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,...,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR,preparation_time,driver_wait_time,delivery_duration,total_delivery_time
0,163923.0,KL09J9N,6.0,10.1,1,NaN,0.0,NaN,NaN,No,...,2024-08-01 05:30:00,84,2024-08-02 05:33:00,0,Work,KL09J9N X 0 X 84,NaN,NaN,NaN,NaN
1,163924.0,H5LGGFX,3.0,8.4,1,NaN,0.0,NaN,NaN,No,...,2024-08-01 05:30:00,78,2024-08-02 05:34:00,0,Home,H5LGGFX X 0 X 78,NaN,NaN,NaN,NaN
2,163925.0,CYLZB6T,4.0,15.0,1,NaN,0.0,NaN,NaN,No,...,2024-08-01 05:30:00,4,2024-08-02 05:35:00,0,Work,CYLZB6T X 0 X 4,NaN,NaN,NaN,NaN
3,163929.0,4YKUKYN,7.0,27.2,1,NaN,0.0,NaN,NaN,No,...,2024-08-01 05:30:00,157,2024-08-02 05:39:00,0,Home,4YKUKYN X 0 X 157,NaN,NaN,NaN,NaN
4,163930.0,WDNU30K,1.0,6.5,1,NaN,0.0,NaN,NaN,No,...,2024-08-01 05:30:00,160,2024-08-02 05:39:00,0,Home,WDNU30K X 0 X 160,NaN,NaN,NaN,NaN


In [15]:
#  Clip or fill negative/invalid times
time_cols_to_clean = ['preparation_time', 'driver_wait_time', 'delivery_duration', 'total_delivery_time']
for col in time_cols_to_clean:
    orders[col] = orders[col].clip(lower=0)
    orders[col] = orders[col].fillna(orders[col].median())

In [17]:
orders.head()

,order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,...,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR,preparation_time,driver_wait_time,delivery_duration,total_delivery_time
0,163923.0,KL09J9N,6.0,10.1,1,NaN,0.0,NaN,NaN,No,...,2024-08-01 05:30:00,84,2024-08-02 05:33:00,0,Work,KL09J9N X 0 X 84,9.0,25.0,16.0,62.0
1,163924.0,H5LGGFX,3.0,8.4,1,NaN,0.0,NaN,NaN,No,...,2024-08-01 05:30:00,78,2024-08-02 05:34:00,0,Home,H5LGGFX X 0 X 78,9.0,25.0,16.0,62.0
2,163925.0,CYLZB6T,4.0,15.0,1,NaN,0.0,NaN,NaN,No,...,2024-08-01 05:30:00,4,2024-08-02 05:35:00,0,Work,CYLZB6T X 0 X 4,9.0,25.0,16.0,62.0
3,163929.0,4YKUKYN,7.0,27.2,1,NaN,0.0,NaN,NaN,No,...,2024-08-01 05:30:00,157,2024-08-02 05:39:00,0,Home,4YKUKYN X 0 X 157,9.0,25.0,16.0,62.0
4,163930.0,WDNU30K,1.0,6.5,1,NaN,0.0,NaN,NaN,No,...,2024-08-01 05:30:00,160,2024-08-02 05:39:00,0,Home,WDNU30K X 0 X 160,9.0,25.0,16.0,62.0


In [20]:
orders['is_favorite'] = orders['is_favorite'].map({True: 1, False: 0, 'Y': 1, 'N': 0, 'yes': 1, 'no': 0}).fillna(0).astype(int)
orders['is_rated'] = orders['is_rated'].map({True: 1, False: 0, 'Y': 1, 'N': 0, 'yes': 1, 'no': 0}).fillna(0).astype(int)

In [21]:
# Basic Feature Aggregation
group_cols = ['customer_id', 'LOCATION_NUMBER', 'vendor_id']
agg_orders = orders.groupby(group_cols).agg({
    'item_count': ['count', 'mean'],
    'grand_total': 'mean',
    'vendor_discount_amount': 'mean',
    'vendor_rating': 'mean',
    'driver_rating': 'mean',
    'preparation_time': 'mean',
    'delivery_duration': 'mean',
    'total_delivery_time': 'mean',
    'is_favorite': 'max',
    'is_rated': 'max',
}).reset_index()

In [45]:
orders.head()

,order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,...,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR,preparation_time,driver_wait_time,delivery_duration,total_delivery_time
0,163923.0,KL09J9N,6.0,10.1,1,NaN,0.0,NaN,0,0,...,2024-08-01 05:30:00,84,2024-08-02 05:33:00,0,Work,KL09J9N X 0 X 84,9.0,25.0,16.0,62.0
1,163924.0,H5LGGFX,3.0,8.4,1,NaN,0.0,NaN,0,0,...,2024-08-01 05:30:00,78,2024-08-02 05:34:00,0,Home,H5LGGFX X 0 X 78,9.0,25.0,16.0,62.0
2,163925.0,CYLZB6T,4.0,15.0,1,NaN,0.0,NaN,0,0,...,2024-08-01 05:30:00,4,2024-08-02 05:35:00,0,Work,CYLZB6T X 0 X 4,9.0,25.0,16.0,62.0
3,163929.0,4YKUKYN,7.0,27.2,1,NaN,0.0,NaN,0,0,...,2024-08-01 05:30:00,157,2024-08-02 05:39:00,0,Home,4YKUKYN X 0 X 157,9.0,25.0,16.0,62.0
4,163930.0,WDNU30K,1.0,6.5,1,NaN,0.0,NaN,0,0,...,2024-08-01 05:30:00,160,2024-08-02 05:39:00,0,Home,WDNU30K X 0 X 160,9.0,25.0,16.0,62.0


In [24]:
# Flatten column names
agg_orders.columns = ['_'.join(col).strip('_') for col in agg_orders.columns.values]

In [ ]:
# Rename for clarity
agg_orders.rename(columns={
    'customer_id_': 'customer_id',
    'LOCATION_NUMBER_': 'location_number',
    'vendor_id_': 'vendor_id',
    'item_count_count': 'num_orders',
    'item_count_mean': 'avg_items_per_order',
    'grand_total_mean': 'avg_spend',
    'vendor_discount_amount_mean': 'avg_discount',
    'vendor_rating_mean': 'avg_vendor_rating',
    'driver_rating_mean': 'avg_driver_rating',
    'preparation_time_mean': 'avg_prep_time',
    'delivery_duration_mean': 'avg_delivery_duration',
    'total_delivery_time_mean': 'avg_total_delivery_time',
    'is_favorite_max': 'was_favorite',
    'is_rated_max': 'was_rated'
}, inplace=True)

In [26]:
# Add label column: 1 for actual interaction
agg_orders['ordered'] = 1

In [27]:
# Save interaction table (will be merged later with negative samples)
agg_orders.head()

,customer_id,LOCATION_NUMBER,vendor_id,num_orders,avg_items_per_order,avg_spend,avg_discount,avg_vendor_rating,avg_driver_rating,avg_prep_time,avg_delivery_duration,avg_total_delivery_time,was_favorite,was_rated,ordered
0,000THBA,0,148,0,NaN,29.60,0.0,0.0,0.0,9.0,16.0,62.0,0,0,1
1,002510Y,0,157,1,3.0,13.60,0.0,NaN,0.0,9.0,16.0,62.0,0,0,1
2,005ECL6,0,237,2,4.5,24.35,0.0,NaN,0.0,8.5,17.0,44.0,0,0,1
3,0075AM7,0,145,1,1.0,25.80,0.0,NaN,0.0,23.0,69.0,103.0,0,0,1
4,009UFS1,0,83,1,1.0,6.80,0.0,NaN,0.0,9.0,16.0,62.0,0,0,1


Generating negative samples

In [28]:
import random

In [29]:
# All unique entities
customers = orders[['customer_id', 'LOCATION_NUMBER']].drop_duplicates()
vendors = orders['vendor_id'].unique()

In [31]:
# Convert actual positives to a set for fast lookup
positive_keys = set(agg_orders.apply(lambda row: (row['customer_id'], row['LOCATION_NUMBER'], row['vendor_id']), axis=1))

In [38]:
# Prepare sets for fast lookup
positive_keys = set(zip(orders['customer_id'], orders['LOCATION_NUMBER'], orders['vendor_id']))
customers = orders[['customer_id', 'LOCATION_NUMBER']].drop_duplicates()
vendors = orders['vendor_id'].unique()
num_neg_per_customer = 2  # or 5

neg_samples = []

In [39]:
for idx, row in customers.iterrows():
    cid = row['customer_id']
    loc = row['LOCATION_NUMBER']
    # Vendors already ordered from at this location
    ordered_vendors = set(orders.loc[
        (orders['customer_id'] == cid) & (orders['LOCATION_NUMBER'] == loc),
        'vendor_id'
    ])
    # Vendors not ordered from
    candidate_neg_vendors = np.setdiff1d(vendors, list(ordered_vendors), assume_unique=True)
    if len(candidate_neg_vendors) == 0:
        continue
    sampled_neg_vendors = np.random.choice(candidate_neg_vendors, 
                                           size=min(num_neg_per_customer, len(candidate_neg_vendors)), 
                                           replace=False)
    for vid in sampled_neg_vendors:
        neg_key = (cid, loc, vid)
        if neg_key not in positive_keys:
            neg_samples.append({
                'customer_id': cid,
                'location_number': loc,
                'vendor_id': vid,
                'ordered': 0
            })

In [40]:
neg_df = pd.DataFrame(neg_samples)

In [41]:
final_data = pd.concat([agg_orders, neg_df], ignore_index=True)

In [42]:
final_data.head()

,customer_id,LOCATION_NUMBER,vendor_id,num_orders,avg_items_per_order,avg_spend,avg_discount,avg_vendor_rating,avg_driver_rating,avg_prep_time,avg_delivery_duration,avg_total_delivery_time,was_favorite,was_rated,ordered,location_number
0,000THBA,0.0,148,0.0,NaN,29.60,0.0,0.0,0.0,9.0,16.0,62.0,0.0,0.0,1,NaN
1,002510Y,0.0,157,1.0,3.0,13.60,0.0,NaN,0.0,9.0,16.0,62.0,0.0,0.0,1,NaN
2,005ECL6,0.0,237,2.0,4.5,24.35,0.0,NaN,0.0,8.5,17.0,44.0,0.0,0.0,1,NaN
3,0075AM7,0.0,145,1.0,1.0,25.80,0.0,NaN,0.0,23.0,69.0,103.0,0.0,0.0,1,NaN
4,009UFS1,0.0,83,1.0,1.0,6.80,0.0,NaN,0.0,9.0,16.0,62.0,0.0,0.0,1,NaN
